<a href="https://colab.research.google.com/github/userNOTfound-bot/Ollama_using_Google-Colab/blob/main/ollama_using_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚠️ You must have ***Ollama*** installed in your system.
If you didn't had that, then follow these steps to install that.

Installation step:
1. go to Ollama official website or use this redirect link https://ollama.com
2. Click on the download button and choose your OS, then download & install.

***Installing Ollama in your system is done.***

In [ ]:
#@title Install ollama on the host
!curl https://ollama.ai/install.sh | sh

In [ ]:
# @title Cloudflared-Tunnel to share Ollama publicly


# Install cloudflared tunnel for public usage & dependencies for python script
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!cloudflared -v

import os
os.environ['OLLAMA_HOST'] = '0.0.0.0'
import asyncio
from aiohttp import ClientSession

# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run(cmd):
  '''
  run is a helper function to run subcommands asynchronously.
  '''
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

# Print lines utf-8 encoded
await asyncio.gather(
    run(['ollama', 'serve']),
    run(['cloudflared', 'tunnel', '--url', 'http://localhost:11434']),
)
